In [ ]:
import torch
import matplotlib
import matplotlib.pyplot as plt
from time import sleep, time
from vizdoom import gymnasium_wrapper
import gymnasium
import random
import math

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Running on device: {device}")
env = gymnasium.make("VizdoomBasic-v0", render_mode="human")

Running on device: cuda:0


c:\Users\killi\AppData\Local\Programs\Python\Python312\Lib\site-packages\vizdoom\gymnasium_wrapper\base_gymnasium_env.py:84: UserWarning: Detected screen format CRCGCB. Only RGB24 and GRAY8 are supported in the Gymnasium wrapper. Forcing RGB24.
  warnings.warn(


In [ ]:
import rlqlearning.rlqlUtil as util
import rlqlearning.DQN as DQN

In [ ]:
BATCH_SIZE = 128 # Transitions sampled from replay buffer
GAMMA = 0.99 # Discount factor
EPSILON_START = 0.9 # Start, end, and rate of decay of the epsilon value 
EPSILON_END = 0.05
EPSILON_DECAY = 1000
TAU = 0.005 # Target network update rate
LEARNING_RATE = 1e-4 # optimiser learning rate

num_actions = env.action_space.n
state = env.reset()
num_obs = len(state)

In [ ]:
policy = DQN(num_obs, num_actions).to(device)
target = DQN(num_obs, num_actions).to(device)
target.load_state_dict(policy.state_dict())

optimiser = torch.optim.AdamW(policy.parameters(), lr=LEARNING_RATE, amsgrad=True)
memory = util.ReplayMemory(10000)

In [ ]:
steps_done = 0

def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPSILON_END + (EPSILON_START - EPSILON_END) * \
        math.exp(-1. * steps_done / EPSILON_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            return policy(state).max(1).indices.view(1, 1)
    else:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)

In [ ]:
for i in range(5):
    done = False
    observation = env.reset()
    score = 0

    while not done:  
        action = env.action_space.sample()
        #print(action)
        #break
        observation, reward, done, trunc, info = env.step(action)
        score += reward
        #print(score)

        env.render()
        #sleep(1/30)

        if done:
            print(f"Episode Score: {score}")